In [ ]:
# Click the play button at the upper left of this block.  
# It should connect to a "runtime" at Google's servers and then run the code in this block.
# If that worked, scroll down and continue on to the next block, below this one.
# If it didn't work (you'll see red lines and text), try to troubleshoot it, and then seek help from your instructor.

# Imports data and time management packages.
# Unfortunately, the time management package does not help in the rest of life :-|
import csv
import pandas as pd
import time
import re

# Installs and imports a package called CIRpy.
# This "resolves" chemicals with help from NIH.
# Basically, if an input is a known chemical name, their servers will recognize it and give you
# a machine readable "structure" called a "SMILES string".
!pip install cirpy
import cirpy

  Created wheel for cirpy: filename=CIRpy-1.0.2-cp37-none-any.whl size=7269 sha256=f61d83afb10158e9ac1737b56e585019038e0596164b1d380d6e4c14cf85a963
  Stored in directory: /root/.cache/pip/wheels/90/31/65/5c9e5a9c65c11466a31aa8360a146cb92ee3abbd628f2801f7
Successfully built cirpy


In [ ]:
# Find a highly processed food or drink or a household product 
# like soap or shampoo.
# Type the name at right where it says "product_name":

# If you can, find a list of the ingredients online, and paste your
# ingredients in the field at right where it just says "Corn Syrup"
# to start with.
# You may need to delete any quotation marks within the list of 
# ingredients for it to work.

# Once you've pasted them in, click the play button at left.

product_name = "fruit roll ups" #@param {type:"string"}
product_name = product_name.replace(' ', '_')

ingredients = "Corn Syrup" #@param {type:"string"}

# These next lines of code clean up some common punctuation and 
# words in ingredients lists that prevent software from
# recognizing chemical names.
# If these fail, your instructor can find help for you.
# In the worst case scenario, you can go back and type in all 
# your ingredients with commas and no other punctuation 
# like "enriched corn meal, corn meal, ferrous sulfate..."
# No worries.


ingredients = ingredients.replace(' [', ', ').replace('and/or ', '').replace('and', '').replace(':', ',')
ingredients = ingredients.replace(']', '').replace(' (', ', ').replace(')', '').replace('.', ',')

print(product_name)
print(ingredients)

fruit_roll_ups
Corn Syrup, Dried Corn Syrup, Sugar, Pear Puree Concentrate, Palm Oil, Contains 2% Or Less Of, Citric Acid, Sodium Citrate, Fruit Pectin, Monoglycerides, Malic Acid, Vitamin C, ascorbic Acid, Acetylated Monoglycerides, Natural Flavor, Color, red 40, Yellows 5 & 6, Blue 1


In [ ]:
# The computer interprets what you input for "ingredients" as one long sentence.
# This code chops it up into a list of individual words by starting a new word 
# every time it sees a comma followed by a space.

ingredients_list = ingredients.split(', ')

print(ingredients_list)

['Corn Syrup', 'Dried Corn Syrup', 'Sugar', 'Pear Puree Concentrate', 'Palm Oil', 'Contains 2% Or Less Of', 'Citric Acid', 'Sodium Citrate', 'Fruit Pectin', 'Monoglycerides', 'Malic Acid', 'Vitamin C', 'ascorbic Acid', 'Acetylated Monoglycerides', 'Natural Flavor', 'Color', 'red 40', 'Yellows 5 & 6', 'Blue 1']


In [ ]:
# This code asks NIH's servers to resolve each ingredient in your list, basically asking if it's a known chemical.
# The time.sleep() line pauses between each request so we don't freak out NIH's servers.
# (Yes, I did that by accident while testing this code.  It's better this way.)

# If the word or phrase is recognized as a chemical, the computer will return a SMILES string.
# These look weird to us, like 'OC(=O)c1cccnc1', but a computer is able to decode them into a chemical structure.

# If the word isn't a recognizable chemical, NIH will return the word "None".
# If you have some "None" values and some SMILES strings, that's okay, just keep going!

# If you only have "None" values and no funky looking SMILES strings like 'OC(=O)c1cccnc1', try starting over with a 
# different list of ingredients from a more highly processed food, then if it still gives all "None", ask for help.

smiles_list = []

for ingredient in ingredients_list:
  try:
    smiles_list.append(cirpy.resolve(ingredient, 'smiles'))
    print(ingredient, smiles_list[-1])
    time.sleep(1.1)
    
  except:
    pass

print('\n')
print(smiles_list)

Corn Syrup None
Dried Corn Syrup None
Sugar OC[C@H]1O[C@H](O[C@]2(CO)O[C@H](CO)[C@@H](O)[C@@H]2O)[C@H](O)[C@@H](O)[C@@H]1O
Pear Puree Concentrate None
Palm Oil None
Contains 2% Or Less Of None
Citric Acid OC(=O)CC(O)(CC(O)=O)C(O)=O
Sodium Citrate [Na+].[Na+].[Na+].OC(CC([O-])=O)(CC([O-])=O)C([O-])=O
Fruit Pectin None
Monoglycerides None
Malic Acid OC(CC(O)=O)C(O)=O
Vitamin C OC[C@H](O)[C@H]1OC(=O)C(=C1O)O
ascorbic Acid OCC(O)C1OC(=C(O)C1=O)O
Acetylated Monoglycerides None
Natural Flavor None
Color None
red 40 None
Yellows 5 & 6 None
Blue 1 None


[None, None, 'OC[C@H]1O[C@H](O[C@]2(CO)O[C@H](CO)[C@@H](O)[C@@H]2O)[C@H](O)[C@@H](O)[C@@H]1O', None, None, None, 'OC(=O)CC(O)(CC(O)=O)C(O)=O', '[Na+].[Na+].[Na+].OC(CC([O-])=O)(CC([O-])=O)C([O-])=O', None, None, 'OC(CC(O)=O)C(O)=O', 'OC[C@H](O)[C@H]1OC(=O)C(=C1O)O', 'OCC(O)C1OC(=C(O)C1=O)O', None, None, None, None, None, None]


In [ ]:
# This code organizes your original list and the new list of chemical SMILES results into a single table, 
# or DataFrame called "df".

# The "dropna()" line says "you know what, if it wasn't a chemical and has "None" for a structure, 
# I don't want it in my table".

# The drop_duplicates() line says "look, I know they sneak 'sugar' in there multiple times,
# sometimes with different names for it, but I only need it in my table once."
# Cleans up nicely, doesn't it?

df = pd.DataFrame(zip(ingredients_list, smiles_list))
print(df.head())
df.dropna(inplace=True)
df.rename(columns={0:'Ingredient', 1:'SMILES String'}, inplace=True)
df.drop_duplicates(subset=('SMILES String'), inplace=True)
print('\n')
print(df.head())

                        0                                                  1
0              Corn Syrup                                               None
1        Dried Corn Syrup                                               None
2                   Sugar  OC[C@H]1O[C@H](O[C@]2(CO)O[C@H](CO)[C@@H](O)[C...
3  Pear Puree Concentrate                                               None
4                Palm Oil                                               None


        Ingredient                                      SMILES String
2            Sugar  OC[C@H]1O[C@H](O[C@]2(CO)O[C@H](CO)[C@@H](O)[C...
6      Citric Acid                         OC(=O)CC(O)(CC(O)=O)C(O)=O
7   Sodium Citrate  [Na+].[Na+].[Na+].OC(CC([O-])=O)(CC([O-])=O)C(...
10      Malic Acid                                  OC(CC(O)=O)C(O)=O
11       Vitamin C                     OC[C@H](O)[C@H]1OC(=O)C(=C1O)O


In [ ]:
# This code saves your data as a "csv", or "comma-separated-value" file in the file browser
# at the left of this window.

df.to_csv('{}_ingredients_names_and_smiles.csv'.format(product_name))

In [ ]:
# Click the folder at the left of your screen, and (once it refreshes) you should
# see your file.  If you don't see it, click the folder icon to minimize the tab, then
# click it again to reopen it.

# Right-click the file to download it to your computer, then drag it from your
# downloads folder to somewhere you want to keep it.
# It should now open with Excel, and you can move on to the next steps...